In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense

In [28]:
data = pd.read_csv('uci-news-aggregator.csv')

In [29]:
# Drop irrelevant columns
data = data[['TITLE', 'CATEGORY']]

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Encode the target variable
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data['CATEGORY'])
test_labels = label_encoder.transform(test_data['CATEGORY'])

# Tokenize and vectorize the headlines
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(train_data['TITLE'])
test_features = vectorizer.transform(test_data['TITLE'])

In [30]:
# Define the model architecture
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(train_features.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
model.fit(train_features, train_labels, batch_size=32, epochs=10, verbose=1)

InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[1] = [0,9729] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse] name: 

In [ ]:
predictions = model.predict(test_features)
predicted_labels = predictions.argmax(axis=1)

In [ ]:
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels, average='weighted')
recall = recall_score(test_labels, predicted_labels, average='weighted')
f1 = f1_score(test_labels, predicted_labels, average='weighted')

In [ ]:
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)